In [22]:
import torch
import torch.nn as nn
import d2l.torch as d2l
import torch.nn.functional as F
import torch.utils.data as data
import torchvision


In [23]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10)
)


In [24]:
batch_size = 256
trans = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
mnist_train = torchvision.datasets.FashionMNIST(
    "./dat/mnist/", True, transform=trans, download=True
)
mnist_test = mnist_train = torchvision.datasets.FashionMNIST(
    "./dat/mnist/", False, transform=trans, download=True
)

train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True)
test_iter = data.DataLoader(mnist_test, batch_size, shuffle=False)


In [47]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)


net.apply(init_weights)

num_epochs, lr = 10, .9
optimizer = torch.optim.SGD(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    metric = d2l.Accumulator(3)
    net.train()
    for i, (X, y) in enumerate(train_iter):
        optimizer.zero_grad()
        y_hat = net(X)
        l = loss(y_hat, y)
        l.backward()
        optimizer.step()


In [48]:
net.eval()
metric = d2l.Accumulator(2)
for X_test,y_test in test_iter:
    y_test_hat = net(X_test)
    metric.add(d2l.accuracy(y_test_hat,y_test),y_test.numel())

metric[0]/metric[1]


0.2